In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import List
import os

# 定义输出数据模型
class ReviewAnalysis(BaseModel):
    sentiment: str = Field(description="情感倾向:积极、消极或中性")
    score: float = Field(description="情感评分,范围0-10")
    pros: List[str] = Field(description="产品优点列表")
    cons: List[str] = Field(description="产品缺点列表")
    summary: str = Field(description="评论总结")
    recommendation: str = Field(description="是否推荐购买:推荐/不推荐/中立")
# 初始化组件
# 移除或禁用可能会导致验证错误的不受支持代理（例如 socks://）
for proxy_var in ("ALL_PROXY", "all_proxy", "HTTP_PROXY", "http_proxy", "HTTPS_PROXY", "https_proxy"):
    os.environ.pop(proxy_var, None)

chat_model  = ChatOpenAI(
    temperature=0.7,
    model="glm-4.6",
    openai_api_key="2a4b3cfb15144a99bd19f7ede356dcd8.FFnDZ27A2JTZVPEk",
    openai_api_base="https://open.bigmodel.cn/api/paas/v4/"
)


parser = PydanticOutputParser(pydantic_object=ReviewAnalysis)

# 创建提示词模板
prompt = ChatPromptTemplate.from_messages([
    ("system", "你是一个专业的产品评论分析专家。你需要客观、准确地分析用户评论。"),
    ("human", """请分析以下产品评论:

{review}

{format_instructions}

请确保分析全面、客观,提取所有重要信息。""")
])

# 构建处理链
def analyze_review(review_text: str) -> ReviewAnalysis:
    """分析产品评论"""
    messages = prompt.format_messages(
        review=review_text,
        format_instructions=parser.get_format_instructions()
    )
    
    response = chat_model.invoke(messages)
    analysis = parser.parse(response.content)
    
    return analysis

# 测试应用
if __name__ == "__main__":
    # 示例评论
    sample_review = """
    这款智能手表真的很不错!屏幕显示清晰,电池续航能力强,可以用3-4天。
    运动追踪功能准确,心率监测也很及时。但是价格稍微有点贵,而且表带材质
    一般,戴久了不太舒服。整体来说,如果预算充足还是值得购买的。
    """
    
    print("正在分析评论...")
    result = analyze_review(sample_review)
    
    print("\n=== 评论分析报告 ===")
    print(f"情感倾向: {result.sentiment}")
    print(f"评分: {result.score}/10")
    print(f"\n优点:")
    for i, pro in enumerate(result.pros, 1):
        print(f"  {i}. {pro}")
    print(f"\n缺点:")
    for i, con in enumerate(result.cons, 1):
        print(f"  {i}. {con}")
    print(f"\n总结: {result.summary}")
    print(f"推荐度: {result.recommendation}")

# 进阶:使用LCEL(LangChain Expression Language)链式调用
from langchain_core.runnables import RunnablePassthrough

chain = (
    {"review": RunnablePassthrough(), 
     "format_instructions": lambda _: parser.get_format_instructions()}
    | prompt
    | chat_model
    | parser
)

# 使用链式调用
result_chain = chain.invoke(sample_review)
print("\n使用LCEL链式调用的结果:")
print(result_chain)

正在分析评论...

=== 评论分析报告 ===
情感倾向: 积极
评分: 8.0/10

优点:
  1. 屏幕显示清晰
  2. 电池续航能力强 (可用3-4天)
  3. 运动追踪功能准确
  4. 心率监测及时

缺点:
  1. 价格稍微有点贵
  2. 表带材质一般，戴久了不舒服

总结: 该用户对智能手表的核心功能，如清晰的屏幕、强大的电池续航（3-4天）以及准确的运动和心率追踪，给予了正面评价。然而，他们也指出了两个缺点：价格偏高以及表带长时间佩戴不舒适。总体而言，用户认为如果预算允许，这款手表还是值得购买的。
推荐度: 推荐

使用LCEL链式调用的结果:
sentiment='积极' score=8.0 pros=['屏幕显示清晰', '电池续航能力强，可用3-4天', '运动追踪功能准确', '心率监测及时'] cons=['价格有点贵', '表带材质一般，戴久了不太舒服'] summary='用户对该智能手表的核心功能（如屏幕显示、电池续航、运动和心率追踪）表示非常满意，认为其表现优秀。主要的负面反馈集中在价格偏高以及表带材质和佩戴舒适度方面。' recommendation='推荐'
